In [9]:
%%capture
!pip install --upgrade transformers
!pip install datasets
!pip install rouge_score
!pip install rouge

In [10]:
import transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, pipeline
from datasets import load_dataset, load_metric, Dataset
import torch
import numpy as np
import pandas as pd
import io
import math
import time

In [11]:
from google.colab import drive

drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab Notebooks/CSC413/amazon_review_dataset_processed.csv"
df = pd.read_csv(path)
amazon = Dataset.from_pandas(df)
amazon.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(11848, 3)

In [12]:
NotTest_Test = amazon.train_test_split(test_size=0.1,seed=42)
NotTest = NotTest_Test["train"]
test = NotTest_Test["test"]

Train_Val = NotTest.train_test_split(test_size=0.1, seed=42)
train = Train_Val["train"]
val = Train_Val["test"]

In [13]:
generator = pipeline(task="summarization", model="flax-community/t5-base-cnn-dm")

In [14]:
reviews = test['reviewText']
summaries = test['summary']

output_batch_size=200
max_summary_length = 25
min_summary_length=2

outputs= generator(reviews[:output_batch_size], max_length=max_summary_length, min_length=min_summary_length, num_beams=2)
generated_summaries = [outputs[i]['summary_text'] for i in range(len(outputs))]

Token indices sequence length is longer than the specified maximum sequence length for this model (1991 > 512). Running this sequence through the model will result in indexing errors


In [15]:
df_result = pd.DataFrame({'review':reviews[:200],'generated_summaries':generated_summaries,'target_summaries':summaries[:200]})
df_result.to_csv('/content/drive/MyDrive/Colab Notebooks/CSC413/zero_shot_t5.csv') 

In [16]:
metric = load_metric('rouge')

def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

calc_rouge_scores(generated_summaries, summaries[:output_batch_size])

{'rouge1': 7.6, 'rouge2': 2.2, 'rougeL': 6.8, 'rougeLsum': 6.8}